    작성일 : 20/02/17
    작성자 : 양희승
    
    작성내용 : 무신사 브랜드 랭킹 크롤링
    

In [2]:
import requests, pymysql, math, re, time
from bs4 import BeautifulSoup

In [3]:
## DB 연결 데이터 
config = {"host":"192.168.0.12", "user":"musinsa", "passwd":"1111", "db":"PR_TEST"}
conn = pymysql.connect(**config)

In [12]:
"""
CREATE TABLE IF NOT EXISTS BRAND_RANK(
    RANK_RANGE      VARCHAR(10),
    RANK_SORT       VARCHAR(10),
    BRD_RANK        VARCHAR(5),
    BRD_CODE        VARCHAR(50),
    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);
"""

'\nCREATE TABLE IF NOT EXISTS BRAND_RANK(\n    RANK_RANGE      VARCHAR(10),\n    RANK_SORT       VARCHAR(10),\n    BRD_RANK        VARCHAR(5),\n    BRD_CODE        VARCHAR(50),\n    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()\n);\n'

In [4]:
## html 파싱
def get_html(url):
    html = ""
    req = requests.get(url)
    
    if req.status_code == 200 :
        html = req.text
        
    return html, req.ok

## Rank Brand SORT URL 가져오기
rank_range = ["1d", "1w", "1m", "1y"]
sort = ["", "sale_cnt", "sale_amt", "pv"]

startTime = time.time()

## 랭킹 기간
for r in rank_range:
## 랭킹 기준
    for s in sort:
        html, req = get_html("https://store.musinsa.com/app/usr/brand_rank?d_cat_cd=&u_cat_cd=&range="\
                             +r+"&sex=&popup=&page=&sort="+s)
        soup = BeautifulSoup(html, 'html.parser')
        print("HTTP Status :", req)

        brand = []

    ## 랭크 종류 입력
        brand.append(["rank_range", r])
        if s == "":
            brand.append(["rank_sort", "pop"])
        elif s == "sale_cnt":
            brand.append(["rank_sort", "cnt"])
        elif s == "sale_amt":
            brand.append(["rank_sort", "amt"])
        else:
            brand.append(["rank_sort", "pv"])

    ## 브랜드 코드, 랭크 가져오기
        for inner in soup.find_all("div", class_="li_inner"):        
            rank = inner.find("p", class_="n-label label-default txt_num_rank").text
            brand.append(["brd_rank", str(rank[:rank.index("위")].strip())])
            brand.append(["brd_code", inner.find("p", class_="brand_name").find("a").get("href")[22:].strip()])

            dict_brand = dict(brand)

        ## DB 적재
            cursor = conn.cursor()
            sql = "INSERT INTO BRAND_RANK(RANK_RANGE, RANK_SORT, BRD_RANK, BRD_CODE) VALUES("\
                 + "'" + dict_brand.get("rank_range", '오류') +"'" +  ","\
                 + "'" + dict_brand.get("rank_sort", '오류') +"'" +  ","\
                 + "'" + dict_brand.get("brd_rank", '오류') +"'" +  ","\
                 + "'" + dict_brand.get("brd_code", '오류') +"'" + ");"

            cursor.execute(sql)
            conn.commit()

        print(r+"_"+dict_brand.get("rank_sort", '오류')+"_완료")

conn.close()
print("--- %s seconds ---",(time.time() - startTime))

HTTP Status : True
1d_pop_완료
HTTP Status : True
1d_cnt_완료
HTTP Status : True
1d_amt_완료
HTTP Status : True
1d_pv_완료
HTTP Status : True
1w_pop_완료
HTTP Status : True
1w_cnt_완료
HTTP Status : True
1w_amt_완료
HTTP Status : True
1w_pv_완료
HTTP Status : True
1m_pop_완료
HTTP Status : True
1m_cnt_완료
HTTP Status : True
1m_amt_완료
HTTP Status : True
1m_pv_완료
HTTP Status : True
1y_pop_완료
HTTP Status : True
1y_cnt_완료
HTTP Status : True
1y_amt_완료
HTTP Status : True
1y_pv_완료
--- %s seconds --- 7.489039421081543
